In [1]:
import json
import os
import pandas as pd

In [2]:
# project_path=${YOUR_PROJECT_PATH}
project_path="/root/SOJUNG_STUFF/ChartMimic"

In [3]:
def get_code_passed_files(modelagent):
    template_type = modelagent.split("_")[-1].split("Agent")[0].lower()
    print(template_type)
    file_dir = project_path + "/results/direct/chart2code_{}_results/{}_checker".format(modelagent, template_type)
    filter_files = os.listdir(file_dir)
    filter_files = [ item.split(".pdf")[0]+".py" for item in filter_files if ".pdf" in item]
    print(len(filter_files))
    return filter_files

In [4]:
# models = ["gpt-4-vision-preview", "claude-3-opus-20240229", "gemini-pro-vision", "Phi-3-vision-128k-instruct", "llava-v1.6-vicuna-7b-hf", "deepseek-vl-7b-chat", "llava-v1.6-mistral-7b-hf", "idefics2-8b", "MiniCPM-Llama3-V-2_5", "Qwen-VL-Chat", "llava-v1.6-vicuna-13b-hf", "cogvlm2-llama3-chat-19B", "InternVL-Chat-V1-5", "llava-v1.6-34b-hf"]
# deepseek-vl-7b-chat
# gemma_nontrained_directcheck
# gemma-trained-direct_checker
# models = ["deepseek-vl-7b-chat"]
models = ["gemma"]
# models = ["gemma_nontrained_directcheck"]
# models = ["gemma-trained-direct_checker"]
# models = ["gemma-trained-v"]
# models = ["deepseek-vl-7b-chat", "gemma_nontrained_directcheck", "gemma-trained-direct_checker"]
agents = ["DirectAgent"]
# agents = ["DirectAgent", "HintEnhancedAgent", "ScaffoldAgent", "SelfRevisionAgent",]

table_type = "all" # "all". "prompting"

model_agents = [ "{}_{}".format(model, agent) for model in models for agent in agents ]

In [ ]:
# filter_type = "no_filter" # no_filter, code_pass, chartedit, different_level
# filter_type = "different_level"
filter_type = "code_pass"
# level = "hard"
denominator = 1800
# denominator = 1800

if filter_type == "no_filter":
    filter_files_dict = None
elif filter_type == "code_pass":
    filter_files_dict = { model_agent: get_code_passed_files(model_agent) for model_agent in model_agents}
else:
    raise ValueError("filter_type not supported")

direct
1135


In [6]:
# construct a dataframe, "model" column is the model name
result_df = pd.DataFrame(columns=["model_agent", "example_count","ExecRate", "TextScore", "LayoutScore", "TypeScore", "ColorScore", "Average", "GPT4VScore", "Overall"])

# insert the model name
result_df["model_agent"] = [ model + "_" + agent for model in models for agent in agents]
# set the index to be the model name
result_df.set_index("model_agent", inplace=True)

In [7]:
files = []
for model in models:
    for agent in agents:
        filename =  project_path + "/results/direct/chart2code_" + model + "_" + agent +"_results_code4evaluation.json"
        if os.path.exists(filename):
            files.append(filename)
        else:
            raise FileNotFoundError("File not found: {}".format(filename))

for idx, file in enumerate(files):
    print("Processing file:", os.path.basename(file))
    
    data = pd.read_json(file, lines=True)
    data["orginial"] = data["orginial"].apply(lambda x: x.split("/")[-1])
    data["generated"] = data["generated"].apply(lambda x: x.split("/")[-1])

    # filter_files = get_code_passed_files(model_agents[idx])
    if filter_files_dict is not None:
        filter_files = filter_files_dict[model_agents[idx]]
        # print(data["orginial"])
        data = data[ data["orginial"].apply(lambda x: any([item == x for item in filter_files])) ]
        print("Length of filter files:", len(filter_files))
        

    print("Length of Data:", len(data))
    print("Denominator:", denominator)
    #print("filter_files:", len(filter_files)) #debug

    f1s = []
    

    result_df.loc[model_agents[idx], "example_count"] = len(data)
    result_df.loc[model_agents[idx], "ExecRate"] = len(filter_files) / denominator
    print("Execution Rate:", len(filter_files) / denominator)


    text_metrics = data["text_metrics"]
    avg_f1 = text_metrics.apply(lambda x: x["f1"]).sum()*100 / denominator
    print(avg_f1)
    result_df.loc[model_agents[idx], "TextScore"] = avg_f1
    f1s.append(avg_f1)

    layout_metrics = data['layout_metrics']
    avg_f1 = layout_metrics.apply(lambda x: x["f1"]).sum()*100 / denominator
    print(avg_f1)
    result_df.loc[model_agents[idx], "LayoutScore"] = avg_f1
    f1s.append(avg_f1)

    chart_type_metrics = data["chart_type_metrics"]
    avg_f1 = chart_type_metrics.apply(lambda x: x["f1"]).sum()*100 / denominator
    print(avg_f1)
    result_df.loc[model_agents[idx], "TypeScore"] = avg_f1
    f1s.append(avg_f1)

    color_metrics = data["color_metrics"]
    avg_f1 = color_metrics.apply(lambda x: x["f1"]).sum()*100 / denominator
    print(avg_f1)
    result_df.loc[model_agents[idx], "ColorScore"] = avg_f1
    f1s.append(avg_f1)

    print( sum(f1s)/len(f1s) )

    result_df.loc[model_agents[idx], "Average"] = sum(f1s)/len(f1s)
    
    zero_f1_rows = data[
    (data["text_metrics"].apply(lambda x: x["f1"]) == 0) &
    (data["layout_metrics"].apply(lambda x: x["f1"]) == 0) &
    (data["chart_type_metrics"].apply(lambda x: x["f1"]) == 0) &
    (data["color_metrics"].apply(lambda x: x["f1"]) == 0)
    ]

    zero_all_f1_count = zero_f1_rows.shape[0]

    print(len(data["text_metrics"].apply(lambda x: x["f1"])))
    print("Number of rows where all 4 f1 scores are zero:", zero_all_f1_count)

    zero_f1_files = zero_f1_rows["orginial"].tolist()
    print("Files where all 4 f1 scores are zero:", zero_f1_files)
    print()

Processing file: chart2code_gemma_DirectAgent_results_code4evaluation.json
Length of filter files: 1135
Length of Data: 891
Denominator: 600
Execution Rate: 1.8916666666666666
95.51633720116666
131.96666666676666
89.07512795578333
78.77657487288333
98.83367667414998
891
Number of rows where all 4 f1 scores are zero: 0
Files where all 4 f1 scores are zero: []



In [8]:
import json
import os

print(model)
input_path = "/root/SOJUNG_STUFF/ChartMimic/highlv_eval/"+ model + "/highlv_results.json"
output_path = "/root/SOJUNG_STUFF/ChartMimic/output_converted_" + model + ".jsonl"  # 확장자 .jsonl 로 바꾸는 게 관례

base_org_dir = "/root/SOJUNG_STUFF/ChartMimic/dataset/direct_600/"
base_gen_dir = "/root/SOJUNG_STUFF/ChartMimic/results/direct/chart2code_"+ model + "_" + agent +"_results/direct_checker"

with open(input_path, "r", encoding="utf-8") as f_in:
    data = json.load(f_in)

with open(output_path, "w", encoding="utf-8") as f_out:
    for entry in data:
        file_name = entry.get("file")
        score = entry.get("score", 0)

        original_path_dir = os.path.join(base_org_dir, file_name)
        generated_path_dir = os.path.join(base_gen_dir, file_name)

        original_path = original_path_dir+ ".py"
        generated_path = generated_path_dir+ ".py"
        reason =  entry.get("reason")
        new_entry = {
            "orginial": original_path,
            "generated": generated_path,
            "gpt4v_score": score,
            "reason": reason
        }

        json_line = json.dumps(new_entry, ensure_ascii=False)
        f_out.write(json_line + "\n")
 

gemma


FileNotFoundError: [Errno 2] No such file or directory: '/root/SOJUNG_STUFF/ChartMimic/highlv_eval/gemma/highlv_results.json'

In [ ]:
files = []
for model in models:
    for agent in agents:
        # /root/SOJUNG_STUFF/ChartMimic/output_converted_gemma_nontrained_directcheck.json
        file =  project_path + "/output_converted_" + model + ".jsonl"
        print(file)
        if os.path.exists(file):
            files.append(file)


for idx, file in enumerate(files):
    print(os.path.basename(file))
    
    data = pd.read_json(file, lines=True)
    data["orginial"] = data["orginial"].apply(lambda x: x.split("/")[-1])
    data["generated"] = data["generated"].apply(lambda x: x.split("/")[-1])

    if filter_files_dict is not None:
        filter_files = filter_files_dict[model_agents[idx]]
        data = data[ data["orginial"].apply(lambda x: any([item == x for item in filter_files])) ]
    print(len(data))


    result_df.loc[model_agents[idx], "example_count"] = len(data)

    gpt4v_score = data["gpt4v_score"]
    avg_gpt4v_score = gpt4v_score.sum() / denominator
    result_df.loc[model_agents[idx], "GPT4VScore"] = avg_gpt4v_score
    print(avg_gpt4v_score)
    
    zero_score_count = (gpt4v_score == 0).sum()
    print("Numbers of gpt4v_score are zero:", zero_score_count)
    zero_score_files = data.loc[data["gpt4v_score"] == 0, "orginial"].tolist()
    print("Files with gpt4v_score == 0:", zero_score_files)
    
        # high-level 결과에서 0점인 행만 필터링
    zero_score_data = data[data["gpt4v_score"] == 0]

    # blank_png 인 파일들만 추출
    blank_png_files = zero_score_data[zero_score_data["reason"] == "blank_png"]["orginial"].tolist()

    # 그 외 다른 이유로 0점 받은 파일들
    non_blank_png_files = zero_score_data[zero_score_data["reason"] != "blank_png"]["orginial"].tolist()

    print()

In [ ]:
# 예시: 두 모델의 0 f1 점수인 파일 리스트
zero_f1_files_model1 = set(zero_f1_files)
zero_f1_files_model2 = set(zero_score_files)

# 교집합: 두 리스트 모두에 포함된 파일
intersection = zero_f1_files_model1 & zero_f1_files_model2

# 차집합: 각 리스트에만 포함된 파일
only_in_model1 = zero_f1_files_model1 - zero_f1_files_model2
only_in_model2 = zero_f1_files_model2 - zero_f1_files_model1

print("❗겹치는 파일 수 (교집합):", len(intersection))
print(sorted(intersection))

print("\n📁low level 에만 있는 파일 수:", len(only_in_model1))
print(sorted(only_in_model1))

print("\n📁high level 에만 있는 파일 수:", len(only_in_model2))
print(sorted(only_in_model2))


In [ ]:
print("high level에서 score 0안 파일들 중...")
# 출력
print("🧊 blank_png reason 파일 수:", len(blank_png_files))
print(sorted(blank_png_files))

print("\n🚫 다른 reason 파일 수:", len(non_blank_png_files))
print(sorted(non_blank_png_files))


In [ ]:
blank_png_files_set = set(
    data[data["gpt4v_score"] == 0]
    .query("reason == 'blank_png'")["orginial"].tolist()
)
print(sorted(blank_png_files_set))
print(len(blank_png_files_set))


files = []
for model in models:
    for agent in agents:
        filename =  project_path + "/results/direct/chart2code_" + model + "_" + agent +"_results_code4evaluation.json"
        if os.path.exists(filename):
            files.append(filename)
        else:
            raise FileNotFoundError("File not found: {}".format(filename))

for idx, file in enumerate(files):
    print("Processing file:", os.path.basename(file))
    
    data = pd.read_json(file, lines=True)
    data["orginial"] = data["orginial"].apply(lambda x: x.split("/")[-1])
    data["generated"] = data["generated"].apply(lambda x: x.split("/")[-1])
    
    if 'blank_png_files_set' in globals():  # 혹시 외부에서 가져온 경우
        def overwrite_f1_if_blank(row):
            if row["orginial"] in blank_png_files_set:
                row["text_metrics"]["f1"] = 0
                row["layout_metrics"]["f1"] = 0
                row["chart_type_metrics"]["f1"] = 0
                row["color_metrics"]["f1"] = 0
            return row
        data = data.apply(overwrite_f1_if_blank, axis=1)

    # filter_files = get_code_passed_files(model_agents[idx])
    if filter_files_dict is not None:
        filter_files = filter_files_dict[model_agents[idx]]
        # print(data["orginial"])
        data = data[ data["orginial"].apply(lambda x: any([item == x for item in filter_files])) ]
        print("Length of filter files:", len(filter_files))
        

    print("Length of Data:", len(data))
    print("Denominator:", denominator)
    #print("filter_files:", len(filter_files)) #debug

    f1s = []
    

    result_df.loc[model_agents[idx], "example_count"] = len(data)
    result_df.loc[model_agents[idx], "ExecRate"] = len(filter_files) / denominator
    print("Execution Rate:", len(filter_files) / denominator)


    text_metrics = data["text_metrics"]
    avg_f1 = text_metrics.apply(lambda x: x["f1"]).sum()*100 / denominator
    print(avg_f1)
    result_df.loc[model_agents[idx], "TextScore"] = avg_f1
    f1s.append(avg_f1)

    layout_metrics = data['layout_metrics']
    avg_f1 = layout_metrics.apply(lambda x: x["f1"]).sum()*100 / denominator
    print(avg_f1)
    result_df.loc[model_agents[idx], "LayoutScore"] = avg_f1
    f1s.append(avg_f1)

    chart_type_metrics = data["chart_type_metrics"]
    avg_f1 = chart_type_metrics.apply(lambda x: x["f1"]).sum()*100 / denominator
    print(avg_f1)
    result_df.loc[model_agents[idx], "TypeScore"] = avg_f1
    f1s.append(avg_f1)

    color_metrics = data["color_metrics"]
    avg_f1 = color_metrics.apply(lambda x: x["f1"]).sum()*100 / denominator
    print(avg_f1)
    result_df.loc[model_agents[idx], "ColorScore"] = avg_f1
    f1s.append(avg_f1)

    print( sum(f1s)/len(f1s) )

    result_df.loc[model_agents[idx], "Average"] = sum(f1s)/len(f1s)
    
    zero_f1_rows = data[
    (data["text_metrics"].apply(lambda x: x["f1"]) == 0) &
    (data["layout_metrics"].apply(lambda x: x["f1"]) == 0) &
    (data["chart_type_metrics"].apply(lambda x: x["f1"]) == 0) &
    (data["color_metrics"].apply(lambda x: x["f1"]) == 0)
    ]

    zero_all_f1_count = zero_f1_rows.shape[0]

    print(len(data["text_metrics"].apply(lambda x: x["f1"])))
    print("Number of rows where all 4 f1 scores are zero:", zero_all_f1_count)

    zero_f1_files = zero_f1_rows["orginial"].tolist()
    print("Files where all 4 f1 scores are zero:", zero_f1_files)
    print()

In [ ]:
# calculate the overall score
result_df["Overall"] = result_df[["Average", "GPT4VScore"]].mean(axis=1)
result_df["ExecRate"] = result_df["ExecRate"] * 100

In [ ]:
result_df

In [ ]:
result_df.to_csv('results.csv', index=True)